In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
# os.chdir("../")


In [10]:
data=pd.read_csv("../merged/len3_ship.csv",encoding="shift-jis",index_col=0)
# whether count or not
count=0
# drop never treated group
data=data.dropna(subset=["release_year"])
data

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,stock_otc_lag,stock_self_lag,Approved,KEGG,Ingredient,date,release_year,成分,variable
3,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,1994.0,D00127,Piroxicam,1995/1/24,1995.0,ピロキシカム,id1
4,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,1994.0,D00132,Ketoprofen,1995/2/10,1995.0,ケトプロフェン,id1
5,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,2002.0,D01578,Pranoprofen,2004/2/16,2004.0,テルビナフィン塩酸塩,id1
6,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1
7,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,2010.0,D01709,Loxoprofen sodium hydrate,2011/1/21,2011.0,ロキソプロフェンナトリウム水和物,id1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,260086.0,260086.0,2007.0,D00222,Aciclovir,2007/10/4,2007.0,アシクロビル,id2
3245,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,260086.0,260086.0,2009.0,D06298,Vidarabine,2009/11/16,2009.0,ビダラビン,id1
3246,2021,629,その他の化学療法剤,56083192,0.0,0,12140098.0,0.0,0.0,18205617.0,0.0,0.0,2002.0,D02219,Terbinafine hydrochloride,2003/10/6,2003.0,プラノプロフェン,id2
3247,2021,629,その他の化学療法剤,56083192,0.0,0,12140098.0,0.0,0.0,18205617.0,0.0,0.0,2011.0,D00282,Clotrimazole,2011/5/10,2011.0,クロトリマゾール,id3


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [11]:
data["elasped"]=data["year"]-data["release_year"]
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
# count ver.
if count:
    aggregated_df = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()
else :
# dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id']).first(), 
        dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
aggregated_df

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,elasped_22,elasped_23,elasped_24,elasped_25,elasped_26,elasped_27,elasped_28,elasped_29,elasped_30,elasped_31
0,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2008,121,局所麻酔剤,12649530,0.0,0,1676687.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2008,123,自律神経剤,5180925,0.0,0,338320.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2008,131,眼科用剤,200817924,43657879.0,0,27228595.0,6132288.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2008,132,耳鼻科用剤,23901015,15040127.0,0,3550000.0,3634742.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2021,441,抗ヒスタミン剤,2350576,1798389.0,470949,724732.0,588014.0,209582.0,929665.0,...,0,0,0,0,0,1,0,0,0,0
360,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,0,1,0,0,0,0,0,0,0
361,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,0,0,0,0,0,0,0,0,0
362,2021,629,その他の化学療法剤,56083192,0.0,0,12140098.0,0.0,0.0,18205617.0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
colnames

['elasped_m14',
 'elasped_m13',
 'elasped_m12',
 'elasped_m11',
 'elasped_m10',
 'elasped_m9',
 'elasped_m8',
 'elasped_m7',
 'elasped_m6',
 'elasped_m5',
 'elasped_m4',
 'elasped_m3',
 'elasped_m2',
 'elasped_m1',
 'elasped_0',
 'elasped_1',
 'elasped_2',
 'elasped_3',
 'elasped_4',
 'elasped_5',
 'elasped_6',
 'elasped_7',
 'elasped_8',
 'elasped_9',
 'elasped_10',
 'elasped_11',
 'elasped_12',
 'elasped_13',
 'elasped_14',
 'elasped_15',
 'elasped_16',
 'elasped_17',
 'elasped_18',
 'elasped_19',
 'elasped_20',
 'elasped_21',
 'elasped_22',
 'elasped_23',
 'elasped_24',
 'elasped_25',
 'elasped_26',
 'elasped_27',
 'elasped_28',
 'elasped_29',
 'elasped_30',
 'elasped_31']

connect generic usage

In [13]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage
# generic from ndb
dgeneric=pd.read_csv("../merged/len3_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric


,薬効分類,year,generic_share_q,generic_share_r
0,114.0,2008.0,0.140940,0.049095
1,114.0,2009.0,0.153114,0.053230
2,114.0,2010.0,0.169573,0.058897
3,114.0,2011.0,0.187411,0.065125
4,114.0,2012.0,0.210339,0.073301
...,...,...,...,...
523,799.0,2019.0,0.006196,0.024777
524,799.0,2020.0,0.006056,0.022847
525,799.0,2021.0,0.007953,0.028184
526,799.0,2022.0,0.010886,0.047744


In [14]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["薬効分類","year"])
merged_df

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,elasped_27,elasped_28,elasped_29,elasped_30,elasped_31,Year,Percentage,薬効分類,generic_share_q,generic_share_r
0,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,114.0,0.140940,0.049095
1,2008,121,局所麻酔剤,12649530,0.0,0,1676687.0,0.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,121.0,0.015019,0.007566
2,2008,123,自律神経剤,5180925,0.0,0,338320.0,0.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,123.0,0.047752,0.026972
3,2008,131,眼科用剤,200817924,43657879.0,0,27228595.0,6132288.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,131.0,0.053601,0.033023
4,2008,132,耳鼻科用剤,23901015,15040127.0,0,3550000.0,3634742.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,132.0,0.013087,0.039617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2021,441,抗ヒスタミン剤,2350576,1798389.0,470949,724732.0,588014.0,209582.0,929665.0,...,1,0,0,0,0,2021,79.00,441.0,0.342768,0.263456
346,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,0,0,0,0,2021,79.00,449.0,0.608051,0.363450
347,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,0,0,0,0,2021,79.00,625.0,0.502607,0.052158
348,2021,629,その他の化学療法剤,56083192,0.0,0,12140098.0,0.0,0.0,18205617.0,...,0,0,0,0,0,2021,79.00,629.0,0.284349,0.073842


In [15]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,id_264,id_265,id_332,id_339,id_399,id_441,id_449,id_625,id_629,id_799
0,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2008,121,局所麻酔剤,12649530,0.0,0,1676687.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2008,123,自律神経剤,5180925,0.0,0,338320.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2008,131,眼科用剤,200817924,43657879.0,0,27228595.0,6132288.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2008,132,耳鼻科用剤,23901015,15040127.0,0,3550000.0,3634742.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2021,441,抗ヒスタミン剤,2350576,1798389.0,470949,724732.0,588014.0,209582.0,929665.0,...,0,0,0,0,0,1,0,0,0,0
346,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,0,0,0,0,0,1,0,0,0
347,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,0,0,0,0,0,0,1,0,0
348,2021,629,その他の化学療法剤,56083192,0.0,0,12140098.0,0.0,0.0,18205617.0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
# dummy ver
if count==0:
    df_with_dummies.to_csv("../merged/len3_ship_agg_drop_never.csv",encoding="shift-jis")
else:
# count ver
    df_with_dummies.to_csv("../merged/len3_ship_sum_agg_drop_never.csv",encoding="shift-jis")